# 5장 : User Logins

 ## Password Hashing

- User model의 password_hash 필드 : 로그인할 때 사용자가 입력한 비밀번호를 검증할 때 사용됨.
- 'Werkzeug' 패키지 : dependency가 있어서 Flask 설치할 때 같이 설치되었음.  
1) generate_password_hash(password) : 해쉬값 생성. 이 해쉬된 비밀번호값으로 원래 패스워드를 알 수는 없음. 같은 값으로 여러 번 생성해도, 그 해쉬 결과는 다름.  
2) check_password_hash(hash, '사용자가 입력한 비밀번호') : true여야 검증완료
- app/models.py 의 User 모델에 password 관련함수 삽입(set_password, check_password_hash)

## Flask-Login에 대한 소개

```bash
pip install flask-login
```

- 'flask-login' 패키지 : 사용자의 로그인 상태를 관리하는 extension. 앱이 사용자의 로그인 상태를 기억하도록 하고, "remember me"라는 기능으로 브라우저 창을 닫은 뒤에도 사용자가 로그인을 유지할 수 있도록 함.
- app/\_\_init\_\_.py 에 LoginManager 관련 코드 추가 : 다른 extension 처럼 Flask-Login도 app/\_\_init\_\_.py에서 앱 인스턴스가 만들어진 다음에, 생성되고 초기화되어야 함.

## Flask-Login을 위해 User Model 준비하기

4개의 아이템만 모델에 추가되면, 어떠한 db 시스템에서도 user model이 동작함
1) is_authenticated : valid credential인지
2) is_active : 사용자 계정이 active 상태인지
3) is_anonymous : 일반적인 유저들은 false
4) get_id() : 사용자에 대한 unique identifier를 String으로 반환하는 함수

- Flask-Login에는 대부분의 user model 클래스에 적합한 일반적인 구현체인 'UserMixin' mixin 클래스를 제공함.
- app/models.py 에 UserMixin 추가

## User Loader 함수

- Flask-Login 은 Flask의 'user session' 내부의 unique identifier들을 저장하고 있음. 로그인된 사용자가 다른 페이지로 가면, Flask-Login은 세션에서 사용자의 ID를 가져와서, 그 사용자를 메모리에 load함. Flask-Login은 db에 대해서 아무것도 모르기 때문에, load할 때 앱의 도움이 필요함. 그래서 extension은 id만 주면 user를 불러오는 user loader 함수를 앱이 정의해뒀을 것으로 기대함.
- @login.user_loader 데코레이터 : user loader함수를 Flask-Login에 등록. 
- app/models.py에 user loader 함수 추가. (쿼리에 id 넣어줄 때는 int로 변환해야)
※ user session : 앱에 연결된 각 사용자들에게 할당된 저장공간. 

## 로그인

- app/routes.py 에서 login 함수 변경
- current_user : Flask-Login에서 온 변수로, 리퀘스트의 클라이언트 user object를 얻을 수 있는 변수.(Flask-Login이 user loader 함수로 얻은 user object거나, 아직 로그인하지 않은 익명의 user object일 수 있음. 로그인 된 사람이 로그인 url로 오면, index 화면으로 보내기 위해, current_user.is_authenticated 변수 사용)
- 입력받은 username과 일치하는 User를 조회해서, 하나만 가져옴.
```python
user = User.query.filter_by(username=form.username.data).first()
```
- 비밀번호 체크. models.py의 User클래스에서 함수 선언을 def check_password(self, password): 이렇게 했고, 부를 때는 u.check_password('입력받은 password') 로 함.
- Flask-Login의 **login_user('user object', remember='')** : 해당 user를 로그인 상태로 등록=다른 페이지에서는 current_user 변수로 그 유저를 가짐.

## 로그아웃

- Flask-Login의 **logout_user()**
- app/routes.py 에 @app.route('/logout') 데코레이터 추가
- app/templates/base.html 에 로그인, 로그아웃 메뉴 추가 및 수정

## 사용자가 로그인하도록 하기

- Flask-Login은 사용자가 앱의 특정 페이지를 보기 전에 반드시 로그인하도록하는 기능이 매우 유용함. 만약 보호된 페이지를 로그인 하지 않은 사용자가 보려고 하면, Flask-Login은 자동으로 사용자를 로그인 폼으로 redirect하고, 로그인 성공해야 사용자가 원했던 페이지로 다시 redirect해줌. 이 기능을 사용하려면, Flask-Login이 로그인을 다루는 view 함수를 알고 있어야 함.
- app/\_\_init\_\_.py 에 login.login_view = 'login' 추가

- @login_required 데코레이터 : view함수의 @app.route 밑에 이 데코레이터를 추가하면, 해당 함수가 보호됨.
- app/routes.py 의 index함수에 상기 데코레이터 추가  

- 사용자가 /index로 접근 > @login_required 데코레이터가 그 요청을 가로채고 /login 으로 redirect하여 응답함. > 하지만 redirect된 url은 /login?next=/index 같은 형태임. > 앱에서 이 next를 받아서, 로그인 후에 해당 url로 다시 redirect해주는 부분은 구현해야함
- app/routes.py 로그인 함수에 리턴 부분 코드 수정
- request.args.get('next') : request.args에서 쿼리스트링을 딕셔너리 형식으로 바꿔줌. 
- login URL에 next argument가
1) 없는 경우 > index로
2) 상대 경로로 들어온 경우 > 그 url로 보내줌.
3) full URL로 들어온 경우(도메인 네임 포함) > index로. 왜냐하면 해커가 next 다음에 유해한 사이트 url을 삽입할 수 있기 때문에, 앱은 url이 상대경로일 때만 redirect해줘야함. 
- url_parse(next_page).netloc : url이 절대 경로인지 상대경로인지 판단하기 위해서 Werkzeug의 'url_parse()' 함수를 사용하고, netloc인지 여부를 판단함.(순수 location이 존재하면=netloc 이 ''이 아니면= 절대 경로임)

## 템플릿에서 로그인된 사용자 보여주기

- Flask-Login 의 current_user 변수를 템플릿에서 사용 (그래서 view function에서 user 변수를 넘겨주지 않아도 됨)

## 사용자 등록

- app/forms.py RegistrationForm 클래스 추가
- validators
1) Email() : 이메일 검증
```bash
pip install email-validator
```
2) EqualTo('password') : 같은지 검증
- validate_<field_name> : WTForms가 뒤의 필드 네임을 보고 커스텀 validator를 만들어서, 기존 validator에 덧붙여서 실행시켜줌. 

- app/templates/register.html 화면 추가
- app/templates/login.html 에 사용자 등록 버튼 추가
- app/routes.py 에 '/register' url 매핑 추가